In [1]:
import torch
from my_data import prepare_input, GamesDataset
from torch.utils.data import DataLoader
import numpy as np

c:\Users\Josh\Anaconda3\envs\team4762\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
print(coordinates)
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    # resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_numbers = [np.flip(np.argsort(prediction)[-5:]) for prediction in predictions]
    # print(resulting_preds_numbers[0])
    # print(predictions[0][resulting_preds_numbers[0][0]])
    # print(predictions[0][resulting_preds_numbers[0][1]])
    # print(predictions[0][resulting_preds_numbers[0][2]])
    # print(predictions[0][resulting_preds_numbers[0][3]])
    # print(predictions[0][resulting_preds_numbers[0][4]])

    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18}


In [3]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")
print(device)

cuda:0


## Save Predictions for Kyu
The same code block applies to Kyu by changing the model and dataset

In [4]:
model = torch.load('./kyu_models/model2300.pth')
model.eval()

df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/kyu_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

test_x_list = []
for game in games:
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    test_x_list.append(test_x)

test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_chars = top_5_preds_with_chars(predictions)
    
    with open('./public_submission.csv','a') as f:
        for index in range(len(prediction_chars)):
            
            answer_row = games_id[idx*32+index] + ',' + ','.join(prediction_chars[index]) + '\n'
            f.write(answer_row)

c:\AIcup_team4762\my_data.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(torch.from_numpy(x), dtype=torch.float32)


11340


## Save Predictions for Dan
The code block below is to use **dan model** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [5]:
model = torch.load('./dan_models/model2000.pth')
model.eval()

df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

test_x_list = []
for game in games:
    
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    test_x_list.append(test_x)


test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_chars = top_5_preds_with_chars(predictions)
    
    with open('./public_submission.csv','a') as f:
        for index in range(len(prediction_chars)):
            answer_row = games_id[idx*32+index] + ',' + ','.join(prediction_chars[index]) + '\n'
            f.write(answer_row)

11000


## Save Predictions for PlayStyle
Playstyle requires only one predictions so the code here is a simple version of above

In [6]:
# Load your own model. Here we use the baseline model
model = torch.load('./playstyle_models/best_PS_model.pth')
model.eval()

df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',1)[-1] for i in df]

pre_data_num=7
test_x_list = []
for game in games:
    move_list = game.split(',')
    test_x = prepare_input(move_list)
    last_idx = len(move_list)-1
    if(last_idx < pre_data_num):
        for i in range(last_idx):
            test_x = torch.cat((test_x, prepare_input(move_list[:last_idx-1-i])[:2]), dim=0)
        for i in range(pre_data_num - last_idx):
            test_x = torch.cat((test_x, torch.zeros((2, 19, 19))), dim=0)      
    else:
        for i in range(pre_data_num):
            test_x = torch.cat((test_x, prepare_input(move_list[:last_idx-1-i])[:2]), dim=0)
   
    test_x_list.append(test_x)


test_x_list = torch.stack(test_x_list)
model = model.to(device)
test_x_list = test_x_list.to(device)

test_dataset = GamesDataset(test_x_list)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


for idx,x in enumerate(data_loader_test):
    predictions=model(x)
    predictions = predictions.cpu()
    predictions = predictions.detach().numpy()
    prediction_numbers = np.argmax(predictions, axis=1)

    with open('./public_submission.csv','a') as f:
        for index, number in enumerate(prediction_numbers):
            answer_row = games_id[idx*32+index] + ',' + str(number+1) + '\n'
            f.write(answer_row)

